### Load graph data

I'm sure there is a way to do this in the java code, but I'm too tired to figure it out right now. For now, just launch a terminal and run the following:
```bash
/opt/gremlin-console/bin/gremlin.sh
```
to connect to the graph service, at the Gremlin console prompt enter:
```bash
g = traversal().withRemote(DriverRemoteConnection.using("127.0.0.1", 8182, "g"));
```
then load the data with:
```bash
g.io("/opt/aerospike-firefly/air-routes-small.graphml").with(IO.reader, IO.graphml).read();
```

### Jupyter setup for Java

In [ ]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;

IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Add dependencies to the `pom.xml`

In [ ]:
%%loadFromPOM
<dependencies>
    <dependency>
        <groupId>org.apache.tinkerpop</groupId>
        <artifactId>gremlin-core</artifactId>
        <version>3.6.4</version>
    </dependency>
    <dependency>
        <groupId>org.apache.tinkerpop</groupId>
        <artifactId>gremlin-driver</artifactId>
        <version>3.6.4</version>
    </dependency>
    <dependency>
        <groupId>org.slf4j</groupId>
        <artifactId>slf4j-api</artifactId>
        <version>2.0.7</version>
    </dependency>
</dependencies>

### Import dependencies

In [ ]:
import org.apache.tinkerpop.gremlin.driver.Cluster;
import org.apache.tinkerpop.gremlin.driver.remote.DriverRemoteConnection;
import org.apache.tinkerpop.gremlin.process.traversal.IO; 
import org.apache.tinkerpop.gremlin.process.traversal.dsl.graph.GraphTraversalSource;
import org.apache.tinkerpop.gremlin.process.traversal.dsl.graph.__;
 
import java.util.Map;
import static org.apache.tinkerpop.gremlin.process.traversal.AnonymousTraversalSource.traversal;

System.out.println("Imports complete.");

### Create and connect to cluster

In [ ]:
private static final String HOST = "127.0.0.1";
private static final int PORT = 8182;
private static final Cluster.Builder BUILDER = Cluster.build().addContactPoint(HOST).port(PORT).enableSsl(false);

System.out.println("Creating the Cluster.");
final Cluster cluster = BUILDER.create();
System.out.println("Creating the GraphTraversalSource.");
final GraphTraversalSource g = traversal().withRemote(DriverRemoteConnection.using(cluster));

### Traverse the data

In [ ]:
// We can use .next() to terminate this traversal because it only returns 1 item  
Map<String, Object> SFO2Hop = g.V().
    has("code", "SFO").
    out().out().dedup().fold().
    project("totalAirportCountFromSFO", "USAirportCountFromSFO").
    by(__.unfold().count()).
    by(__.unfold().has("country", "US").count()).
    next();
System.out.println(SFO2Hop);
cluster.close();